## <b>使用深度学习技术预测股票价格</b>

版本 v1.0

## <b>目录</b>
* ### <b>深度学习策略的交易规则</b>

* ### <b>策略构建步骤</b>

* ### <b>策略的实现</b>

## <b>正文</b>

### <b>一、深度学习策略的交易规则</b>

* 买入条件：预测的上涨概率>0.5，则买入或保持已有持仓。
* 卖出条件 :预测的上涨概率<0.5，则卖出已有股票。

### <b>二、策略构建步骤</b>
#### 1、确定股票池和数据起止时间

* 在证券代码列表m24模块中输入要回测的单只股票，以及数据的起止日期(含训练集和验证集)。

#### 2、确定因子

* 在输入特征列表m8模块中输入用于预测的N个因子表达式。

#### 3、获取基础数据

* 通过自定义模块m23获取指定股票池的基础数据，如收盘价等字段。

#### 4、确定并计算模型标注

* 通过自定义模块m16计算需要的标注指标，本例中首先计算未来10天收益df['return']，然后根据df['return']的正负来给每日数据标注1或0，来标识涨跌。

#### 5、抽取因子数据

* 通过衍生数据抽取模块m26计算因子数据。

#### 6、合并标注与因子数据

* 通过连接数据m17模块合并因子数据和标注数据。

#### 7、划分训练集和预测集

* 通过数据过滤模块m19和m20中设置的日期范围划分训练集数据和预测集数据。

#### 8、生成序列窗口滚动数据集

* 通过序列窗口滚动(深度学习)模块将训练集和预测集的数据生成固定窗口长度的数据序列，为后续模型训练和预测做准备。


#### 9、构建LSTM + CNN模型构架

* 在画布左侧模块列表中依次拖入输入层模块、Reshape层模块、Conv2D层模块、Reshape层模块、LSTM层模块、Dropout层模块和全连接层模块(两组)，构成深度学习网络构架，

  最后通过“构建(深度学习)”模块组装各层。这里需要注意：
    
    输入层的shape参数是 窗口滚动数据集的大小 X 因子数量 , 本例为 50 行 X 5个因子

    ReShape层的参数是 窗口滚动数据集的大小 X 因子数量 X 1 ，本例为 50 行 X 5个因子 X1
    
    Conv2D层中的 kernel_size参数是滑动窗口的尺寸，本例中使用 3行 X 5列 的窗口, 每次滑动的步长为 1行 X 1列 ， 卷积核数目为32，这里的窗口设置决定了后面ReShape层的参数
    
    ReShape层中的target_shape 参数，这是由 窗口滚动数据集 X 因子数量 和 Conv2D层中设置的窗口尺寸以及步长决定的。本例中 50行 X 5因子 的输入数据，使用 3行 X5列 的窗口滑动取数据，
    
    每次移动1行，共计可以得到48次数据(即可以通过滑动3行 X 5列的窗口48次来获取完整的数据)，因此target_shape= 48 X 卷积核数32
    
    LSTM层的输出空间维度设置为卷积核数32，并设置激活函数
    
    Dropout层是防止过度拟合采用的主动裁剪数据技术，这里设置rate 为0.8
    
    全连接层共两层，第一层的输出空间维度与LSTM的输出维度保持一致为32，第二层将第一层的32维数据转变为1维数据输出，即获取预测的label值，此例为0到1之间的连续值，可以认为是上涨的概率。
    

#### 10、训练深度学习模型

* 在画布左侧模块列表中拖入“训练(深度学习)”模块m6,设置属性中的优化器、目标函数、评估指标、每次训练的数据量batch_size、迭代次数epochs和GPU的数量以及日志输出频率。

#### 11、使用深度学习模型预测

* 在画布左侧模块列表中拖入“预测(深度学习)”模块m7，并将“训练(深度学习)”模块m6的模型输出和验证集的序列窗口滚动数据集传给预测模块，通过预测模块即根据股票验证集的数据预测上涨的概率。

#### 12、根据模型预测结果构建策略

* 如果当日预测的上涨概率大于0.5，则保持持仓或买入

* 如果当日预测的上涨概率小于0.5，则卖出股票或保持空仓。

#### 13、模拟回测

* 通过 trade 模块中的初始化函数定义交易手续费和滑点，通过 context.prediction 获取每日的上涨概率预测结果；

* 通过 trade 模块中的主函数(handle函数)查看每日的买卖交易信号，按照买卖原则执行相应的买入/卖出操作。

### <b>三、策略的实现</b>
可视化策略实现如下：

In [4]:
# 本代码由可视化策略环境自动生成 2021年8月22日14:38
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m23_run_bigquant_run(input_1, input_2, input_3):
    fields = ['open','high','low','close','volume']
    input_1_df = input_1.read_pickle()
    ins = input_1_df['instruments']
    start_date = input_1_df['start_date']
    end_date = input_1_df['end_date']
    df = D.history_data(ins, start_date, end_date, fields)     
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m23_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m16_run_bigquant_run(input_1, input_2, input_3):
    input_ds = input_1
    df = input_ds.read_df()
    df['return'] = (df.close.shift(-10)/df.close - 1)
    df['label'] = np.where(df['return'] > 0, 1, 0)
    ds = DataSource.write_df(df[['date','instrument','label']])
    return Outputs(data_1=ds)


# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m16_post_run_bigquant_run(outputs):
    return outputs

# 用户的自定义层需要写到字典中，比如
# {
#   "MyLayer": MyLayer
# }
m6_custom_objects_bigquant_run = {
    
}

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m2_run_bigquant_run(input_1, input_2, input_3):
    import numpy as np
    input_series = input_1
    input_df = input_2
    test_data = input_df.read_pickle()
    pred_label = input_series.read_pickle()
  
    pred_result = pred_label.reshape(pred_label.shape[0]) 
    dt = input_3.read_df()['date'][-1*len(pred_result):]
    pred_df = pd.Series(pred_result, index=dt)
    ds = DataSource.write_df(pred_df)
    
    pred_label = np.where(pred_label>0.5,1,0)
    labels = test_data['y']
    print('准确率%s'%(np.mean(pred_label==labels)))
    
    return Outputs(data_1=ds)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m2_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m1_initialize_bigquant_run(context):
    # 加载预测数据
    context.prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
# 回测引擎：每日数据处理函数，每天执行一次
def m1_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    try:
        prediction = context.prediction[data.current_dt.strftime('%Y-%m-%d')]
    except KeyError as e:
        return
    
    instrument = context.instruments[0]
    sid = context.symbol(instrument)
    cur_position = context.portfolio.positions[sid].amount
    
    # 交易逻辑
    if prediction > 0.5 and cur_position == 0:
        context.order_target_percent(context.symbol(instrument), 1)
        print(data.current_dt, '买入！')
        
    elif prediction < 0.5 and cur_position > 0:
        context.order_target_percent(context.symbol(instrument), 0)
        print(data.current_dt, '卖出！')
    
# 回测引擎：准备数据，只执行一次
def m1_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m1_before_trading_start_bigquant_run(context, data):
    pass


m3 = M.dl_layer_input.v1(
    shape='50,5',
    batch_shape='',
    dtype='float32',
    sparse=False,
    name=''
)

m13 = M.dl_layer_reshape.v1(
    inputs=m3.data,
    target_shape='50,5,1',
    name=''
)

m14 = M.dl_layer_conv2d.v1(
    inputs=m13.data,
    filters=32,
    kernel_size='3,5',
    strides='1,1',
    padding='valid',
    data_format='channels_last',
    dilation_rate='1,1',
    activation='relu',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m15 = M.dl_layer_reshape.v1(
    inputs=m14.data,
    target_shape='48,32',
    name=''
)

m4 = M.dl_layer_lstm.v1(
    inputs=m15.data,
    units=32,
    activation='tanh',
    recurrent_activation='hard_sigmoid',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    recurrent_initializer='Orthogonal',
    bias_initializer='Ones',
    unit_forget_bias=True,
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    recurrent_regularizer='None',
    recurrent_regularizer_l1=0,
    recurrent_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    recurrent_constraint='None',
    bias_constraint='None',
    dropout=0,
    recurrent_dropout=0,
    return_sequences=False,
    implementation='0',
    name=''
)

m11 = M.dl_layer_dropout.v1(
    inputs=m4.data,
    rate=0.8,
    noise_shape='',
    name=''
)

m10 = M.dl_layer_dense.v1(
    inputs=m11.data,
    units=32,
    activation='tanh',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m12 = M.dl_layer_dropout.v1(
    inputs=m10.data,
    rate=0.8,
    noise_shape='',
    name=''
)

m9 = M.dl_layer_dense.v1(
    inputs=m12.data,
    units=1,
    activation='sigmoid',
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='Zeros',
    kernel_regularizer='None',
    kernel_regularizer_l1=0,
    kernel_regularizer_l2=0,
    bias_regularizer='None',
    bias_regularizer_l1=0,
    bias_regularizer_l2=0,
    activity_regularizer='None',
    activity_regularizer_l1=0,
    activity_regularizer_l2=0,
    kernel_constraint='None',
    bias_constraint='None',
    name=''
)

m5 = M.dl_model_init.v1(
    inputs=m3.data,
    outputs=m9.data
)

m8 = M.input_features.v1(
    features="""(close/shift(close,1)-1)*10
(high/shift(high,1)-1)*10
(low/shift(low,1)-1)*10
(open/shift(open,1)-1)*10
(volume/shift(volume,1)-1)*10"""
)

m24 = M.instruments.v2(
    start_date='2015-01-01',
    end_date='2020-01-01',
    market='CN_STOCK_A',
    instrument_list='600009.SHA',
    max_count=0
)

m23 = M.cached.v3(
    input_1=m24.data,
    run=m23_run_bigquant_run,
    post_run=m23_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m16 = M.cached.v3(
    input_1=m23.data_1,
    run=m16_run_bigquant_run,
    post_run=m16_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m26 = M.derived_feature_extractor.v3(
    input_data=m23.data_1,
    features=m8.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m17 = M.join.v3(
    data1=m16.data_1,
    data2=m26.data,
    on='date,instrument',
    how='inner',
    sort=True
)

m21 = M.dropnan.v2(
    input_data=m17.data
)

m19 = M.filter.v3(
    input_data=m21.data,
    expr='date<\'2017-03-01\'',
    output_left_data=False
)

m25 = M.dl_convert_to_bin.v2(
    input_data=m19.data,
    features=m8.data,
    window_size=50,
    feature_clip=5,
    flatten=False,
    window_along_col='instrument'
)

m6 = M.dl_model_train.v1(
    input_model=m5.data,
    training_data=m25.data,
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics='accuracy',
    batch_size=2048,
    epochs=10,
    custom_objects=m6_custom_objects_bigquant_run,
    n_gpus=1,
    verbose='1:输出进度条记录'
)

m20 = M.filter.v3(
    input_data=m21.data,
    expr='date>\'2017-03-01\'',
    output_left_data=False
)

m27 = M.dl_convert_to_bin.v2(
    input_data=m20.data,
    features=m8.data,
    window_size=50,
    feature_clip=5,
    flatten=False,
    window_along_col='instrument'
)

m7 = M.dl_model_predict.v1(
    trained_model=m6.data,
    input_data=m27.data,
    batch_size=10240,
    n_gpus=2,
    verbose='2:每个epoch输出一行记录'
)

m2 = M.cached.v3(
    input_1=m7.data,
    input_2=m27.data,
    input_3=m20.data,
    run=m2_run_bigquant_run,
    post_run=m2_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m1 = M.trade.v4(
    instruments=m24.data,
    options_data=m2.data_1,
    start_date='2020-01-02',
    end_date='',
    initialize=m1_initialize_bigquant_run,
    handle_data=m1_handle_data_bigquant_run,
    prepare=m1_prepare_bigquant_run,
    before_trading_start=m1_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-08-22 14:38:09.004511] INFO: moduleinvoker: dl_layer_input.v1 运行完成[0.00193s].

[2021-08-22 14:38:09.022955] INFO: moduleinvoker: dl_layer_reshape.v1 运行完成[0.015439s].

[2021-08-22 14:38:09.039447] INFO: moduleinvoker: dl_layer_conv2d.v1 运行完成[0.013485s].

[2021-08-22 14:38:09.050245] INFO: moduleinvoker: dl_layer_reshape.v1 运行完成[0.008251s].

[2021-08-22 14:38:09.212530] INFO: moduleinvoker: dl_layer_lstm.v1 运行完成[0.159166s].

...

DataSource(468cd8d1cbd64adfbb91f5feef2a4c03T)


[2021-08-22 14:38:09.441429] INFO: moduleinvoker: dl_model_predict.v1 运行完成[0.005366s].

[2021-08-22 14:38:09.445794] INFO: moduleinvoker: cached.v3 开始运行..

[2021-08-22 14:38:09.501247] ERROR: moduleinvoker: module name: cached, module version: v3, trackeback: TypeError: 'numpy.dtype' object is not iterable\n

TypeError: 'numpy.dtype' object is not iterable